# PostGIS on Amazon RDS testing in Amazon SageMaker Studio
## Ryan Prasad 9/27/2023

![Points Visualized](images/points_vis.png "Points Visualized")

Looks like SageMaker Studio has finally fired up. I got my 80k labeled points and continent polygons loaded up on PostGIS on RDS and now I guess this is the fun part. First thing's first, I need to get this thing to talk to RDS. Last night, I was playing around with this in a regular SageMaker Notebook, and since it was in the same VPC and Security Group as RDS, I was able to get a connection without much fuss.

SageMaker Studio is a bit different. This seems to be partitioned off from the rest of AWS and the traffic goes out to the internet. So I need to grab my IP address and add it to the inbound rules for the Security Group so that I can get connected to RDS and pull that sweet, sweet geospatial data from it.

In [7]:
from requests import get
ip = get('https://api.ipify.org').content.decode ('utf8') 
print ('My public IP address is: {}'.format (ip))

My public IP address is: 34.213.171.250


![Security Group Inbound Rules](images/inbound_rules.png "Security Group Inbound Rules")

Time to get connected and start poking around!

In [12]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

engine = create_engine(
    "{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}".format(
        dialect="postgresql",
        driver="psycopg2",
        username="postgres",
        password="postgres",
        host="fastdb.com40arouubf.us-west-2.rds.amazonaws.com",
        port=5432,
        database="geoml"
    )
)

with engine.connect() as db_conn:
    sql_query = "SELECT COUNT(*) FROM public.continents"
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)

In [13]:
df

,count
0,8


Looks like I have a connection and I also have 8 continents. Lets look at 10 points.

In [18]:
with engine.connect() as db_conn:
    sql_query = "SELECT * FROM public.points_xy ORDER BY id ASC LIMIT 10"
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)
df

,id,geom,cid
0,1,0101000020E6100000E0B4617AE6982D40C0D9B3AF8860...,1
1,2,0101000020E6100000402327D98BEF3E400024E42D9E84...,1
2,3,0101000020E6100000F0D4F6F8827C3E40002B89D02E6F...,1
3,4,0101000020E6100000E08804F35A392D40308F9C75DB4F...,1
4,5,0101000020E610000060DB7D9680EA314090BDEB012D61...,1
5,6,0101000020E6100000A0CE97B6B6D03B4020F190226522...,1
6,7,0101000020E610000000008E17F45D404040B9E9E81B3B...,1
7,8,0101000020E610000000475410520B13C040BE458003CC...,1
8,9,0101000020E610000080AB9CDC402C0FC0780C37696528...,1
9,10,0101000020E6100000881214493833424080CE3CFDC342...,1


So this is behaving as intended. The geom column is the hex representation of the lat/lon in WGS84. cid is the continent ID, which is the Primary Key of the continents table/polygon. I'll make a one to many relation between continents and points_xy in pgAdmin.

![Entity Relationship Diagram](images/erd.png "Entity Relationship Diagram")

Let's swap out the cid for the actual continent name:

In [20]:
with engine.connect() as db_conn:
    sql_query = """
    SELECT public.points_xy.id, public.points_xy.geom, public.continents.continent 
    FROM public.points_xy
    INNER JOIN public.continents ON public.points_xy.cid=public.continents.id
    ORDER BY id ASC LIMIT 10;
    """
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)
df

,id,geom,continent
0,1,0101000020E6100000E0B4617AE6982D40C0D9B3AF8860...,Africa
1,2,0101000020E6100000402327D98BEF3E400024E42D9E84...,Africa
2,3,0101000020E6100000F0D4F6F8827C3E40002B89D02E6F...,Africa
3,4,0101000020E6100000E08804F35A392D40308F9C75DB4F...,Africa
4,5,0101000020E610000060DB7D9680EA314090BDEB012D61...,Africa
5,6,0101000020E6100000A0CE97B6B6D03B4020F190226522...,Africa
6,7,0101000020E610000000008E17F45D404040B9E9E81B3B...,Africa
7,8,0101000020E610000000475410520B13C040BE458003CC...,Africa
8,9,0101000020E610000080AB9CDC402C0FC0780C37696528...,Africa
9,10,0101000020E6100000881214493833424080CE3CFDC342...,Africa


That hex looks kind of gnarly and won't help me much when these need to go into numpy arrays. Let's see if we can get floats from PostGIS. 

In [21]:
with engine.connect() as db_conn:
    sql_query = """
    SELECT public.points_xy.id, ST_AsText(public.points_xy.geom), public.continents.continent 
    FROM public.points_xy
    INNER JOIN public.continents ON public.points_xy.cid=public.continents.id
    ORDER BY id ASC LIMIT 10;
    """
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)
df

,id,st_astext,continent
0,1,POINT(14.798633408000057 10.688542834000032),Africa
1,2,POINT(30.93572766500006 -0.797438706999969),Africa
2,3,POINT(30.486373482000033 -15.717154041999947),Africa
3,4,POINT(14.612022013000058 -16.311942434999935),Africa
4,5,POINT(17.916024595000067 22.379593010000065),Africa
5,6,POINT(27.815287983000076 8.567177849000075),Africa
6,7,POINT(32.73401159700006 5.057723655000075),Africa
7,8,POINT(-4.761055235999947 12.898464211000032),Africa
8,9,POINT(-3.896608088999926 32.31559481800008),Africa
9,10,POINT(36.40015519600007 -14.630401528999982),Africa


Well, that's not exactly what I wanted but it will do for now. When I generated the points in ArcGIS I threw in the lat/lon as additional attributes, but decided to drop those columns since they would remain unchanged if the points were to change in a live production environment, God forbid.

Let's play with some spatial selections. How many points are in Africa according to the continent id?

In [27]:
with engine.connect() as db_conn:
    sql_query = """
    SELECT COUNT(*)
    FROM public.points_xy
    WHERE public.points_xy.cid=1;
    """
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)
df

,count
0,10000


How many points are in the actual Africa polygon, as opposed to the continent id?

In [28]:
with engine.connect() as db_conn:
    sql_query = """
    SELECT COUNT(public.points_xy.*)
    FROM public.points_xy, public.continents
    WHERE ST_contains((SELECT public.continents.geom WHERE public.continents.id=1), public.points_xy.geom);
    """
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)
df

,count
0,10000


Well that's reassuring. Now it's time to make a model! Looks like scikit-learn will do just fine, but first I need to get that well-known text (WKT) for the lat/lon into actual floats and then into NumPy arrays. 

Whenever I get this deployed, I am going to have to hit the database for the continent name. Dunno how that is going to work quite yet...